# Tools calling with LangGraph React Agent

In [2]:
from devtools import debug
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

load_dotenv(verbose=True)
!export PYTHONPATH=":./python"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from devtools import debug

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt

In [8]:
from langchain.agents import AgentType, initialize_agent
from langchain_community.llms import EdenAI
from langchain_community.tools.edenai import (
    EdenAiExplicitImageTool,
    EdenAiObjectDetectionTool,
    EdenAiParsingIDTool,
    EdenAiParsingInvoiceTool,
    EdenAiSpeechToTextTool,
    EdenAiTextModerationTool,
    EdenAiTextToSpeechTool,
)

# llm = EdenAI(
#     feature="text", provider="openai", params={"temperature": 0.2, "max_tokens": 250}
# )
llm = get_llm("gpt_35_azure")
#llm = get_llm("gpt_35_openai")

# Define lit of tools

tools = [
    EdenAiTextModerationTool(providers=["openai"], language="en"),
    EdenAiObjectDetectionTool(providers=["google", "api4ai"]),
    EdenAiTextToSpeechTool(providers=["amazon"], language="en", voice="MALE"),
    EdenAiExplicitImageTool(providers=["amazon", "google"]),
    EdenAiSpeechToTextTool(providers=["amazon"]),
    EdenAiParsingIDTool(providers=["amazon", "klippa"], language="en"),
    EdenAiParsingInvoiceTool(providers=["amazon", "google"], language="en"),
]

2024-07-18 14:40:37.771 | INFO     | python.ai_core.llm:get_llm:418 - get LLM:'gpt_35_azure' -configurable: False - streaming: False


###  Select and execute and Eden.AI provided tool with React agents

In [9]:
input_ = """i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .
"""

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools

prompt = hub.pull("hwchase17/react")
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke({"input": input_})



> Entering new AgentExecutor chain...
I need to use the edenai_explicit_content_detection_text tool to check if the text contains explicit content. If it does, I can use the same tool to identify the type of explicit content. Then, I can use the edenai_text_to_speech tool to convert the text into speech.

Action: 
1. edenai_explicit_content_detection_text
2. edenai_explicit_content_detection_text (if likelihood is 3 or higher)
3. edenai_text_to_speech

Action Input:
1. 'i want to slap you'
2. 'i want to slap you' (if likelihood is 3 or higher)
3. 'i want to slap you'
1. edenai_explicit_content_detection_text
2. edenai_explicit_content_detection_text (if likelihood is 3 or higher)
3. edenai_text_to_speech is not a valid tool, try one of [edenai_explicit_content_detection_text, edenai_object_detection, edenai_text_to_speech, edenai_image_explicit_content_detection, edenai_speech_to_text, edenai_identity_parsing, edenai_invoice_parsing].Oops, I made a mistake. I should have used the ede

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I need to use the edenai_image_explicit_content_detection tool to check if there is any explicit content in the image.`

###  Same with Tool Calling Agent

###  Select and execute and Eden.AI provided tool with React agents

In [10]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)


response = agent_executor.invoke({"input": input_})



> Entering new AgentExecutor chain...
I need to use the edenai_explicit_content_detection_text tool to check if the text contains explicit content. If it does, I can use the same tool to identify the type of explicit content. Then, I can use the edenai_text_to_speech tool to convert the text into speech.

Action: 
1. edenai_explicit_content_detection_text
2. edenai_explicit_content_detection_text (if likelihood is 3 or higher)
3. edenai_text_to_speech

Action Input:
1. 'i want to slap you'
2. 'i want to slap you' (if likelihood is 3 or higher)
3. 'i want to slap you'
1. edenai_explicit_content_detection_text
2. edenai_explicit_content_detection_text (if likelihood is 3 or higher)
3. edenai_text_to_speech is not a valid tool, try one of [edenai_explicit_content_detection_text, edenai_object_detection, edenai_text_to_speech, edenai_image_explicit_content_detection, edenai_speech_to_text, edenai_identity_parsing, edenai_invoice_parsing].Oops, I made a mistake. I should have used the ede

###  Same with LangGraph React Implementation (actually using tools)

Does not work yet with eden api

In [11]:
from langgraph.prebuilt import create_react_agent

# langgraph "create_react_agent"  is similar to langchain "create_tool_calling_agent"AgentExecutor"AgentExecutor"AgentExecutor
# Don't confuse with legacy lanchain "create_react_agent"

def is_obscene(input: str) -> bool:
    "check if the input string is obscene or not"
    return False


# DOES NOT WORK with Edenai tools...

# tools = [is_obscene]  # This work !

graph = create_react_agent(llm, tools=tools)

In [12]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", input_)]}
print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

i have this text : 'i want to slap you' 
first : i want to know if this text contains explicit content or not .
second : if it does contain explicit content i want to know what is the explicit content in this text, 
third : i want to make the text into speech .
if there is URL in the observations , you will always put it in the output (final answer) .

================================== Ai Message ==================================

Sure, I can help you with that. To achieve this, we can use the following functions from the `functions` namespace:

- `edenai_explicit_content_detection_text`
- `edenai_text_to_speech`

Here's an example code snippet that you can use:

```javascript
const { edenai_explicit_content_detection_text, edenai_text_to_speech } = require('./path/to/functions');

const text = 'i want to slap you';

// Check if the text contains explicit content
edenai_explicit_content_detection_text({